# Chinle Primary Care Scheduler
**In this notebook, we'll walk through how to use the scheduler to make the internal medicine scheudle for August 2025.**

In [1]:
import sys
from pathlib import Path

# Add project root (one level up from 'tutorial') to sys.path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
import pandas as pd
from datetime import date
from utils.parser import parse_inputs
from utils.calendar import generate_clinic_calendar
from constraints.internal_medicine import (
    create_shift_variables,
    add_leave_constraints,
    add_inpatient_block_constraints,
    add_rdo_constraints,
    add_clinic_count_constraints,
    add_min_max_staffing_constraints
)
from ortools.sat.python import cp_model

**The first step is to import and clean the YML file and CSV files.**

In [3]:
config, leave_df, inpatient_days_df, inpatient_starts_df = parse_inputs('../config/internal_medicine.yml',
                                                                        '../data/leave_requests.csv',
                                                                        '../data/inpatient.csv')

**Let's inspect the parsed YML file and CSV files.**

In [4]:
config['clinic_rules']

{'clinic_days': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
 'clinic_sessions': {'Monday': ['morning', 'afternoon'],
  'Tuesday': ['morning', 'afternoon'],
  'Wednesday': ['morning', 'afternoon'],
  'Thursday': ['afternoon'],
  'Friday': ['morning', 'afternoon']},
 'random_day_off': {'eligible_days': ['Monday',
   'Tuesday',
   'Wednesday',
   'Friday']},
 'inpatient_schedule': {'inpatient_length': 7,
  'inpatient_starts_on': 'Tuesday',
  'inpatient_ends_on': 'Monday',
  'pre_inpatient_leave': 'Monday',
  'post_inpatient_leave': 'Friday'},
 'staffing': {'min_providers_per_session': 3, 'max_providers_per_session': 5},
 'holiday_dates': [datetime.date(2025, 1, 1),
  datetime.date(2025, 1, 20),
  datetime.date(2025, 2, 17),
  datetime.date(2025, 5, 26),
  datetime.date(2025, 6, 19),
  datetime.date(2025, 7, 4),
  datetime.date(2025, 9, 1),
  datetime.date(2025, 10, 13),
  datetime.date(2025, 11, 11),
  datetime.date(2025, 11, 27),
  datetime.date(2025, 12, 25)],
 'clinic_int

In [21]:
print(leave_df.dtypes)
leave_df.head(5)

provider            object
date        datetime64[ns]
dtype: object


,provider,date
0,Orcutt,2025-08-04
1,Orcutt,2025-08-05
2,Orcutt,2025-08-06
3,Orcutt,2025-08-07
4,Orcutt,2025-08-08


In [6]:
print(inpatient_days_df.dtypes)
inpatient_days_df.head(5)

,provider,date
0,Bornstein,2025-08-05
1,Bornstein,2025-08-06
2,Bornstein,2025-08-07
3,Bornstein,2025-08-08
4,Bornstein,2025-08-09


In [7]:
print(inpatient_starts_df.dytypes)
inpatient_starts_df.head(5)

,provider,start_date
0,Bornstein,2025-08-05
1,Wadlin,2025-08-19
2,Miles,2025-08-26


**The files look well formated and have the correct data types. Let's build our calendar for August 2025. This will only include valid clinic days (ie., not federal holidays) and valid sessions (ie., no Thursday morning).**

In [8]:
calendar = generate_clinic_calendar(date(2025, 8, 4), 
                                    date(2025, 8, 29), 
                                    config['clinic_rules'])

In [9]:
# Here are all the possible clinic sessions that a provider could work in August. 
calendar

{datetime.date(2025, 8, 4): ['morning', 'afternoon'],
 datetime.date(2025, 8, 5): ['morning', 'afternoon'],
 datetime.date(2025, 8, 6): ['morning', 'afternoon'],
 datetime.date(2025, 8, 7): ['afternoon'],
 datetime.date(2025, 8, 8): ['morning', 'afternoon'],
 datetime.date(2025, 8, 11): ['morning', 'afternoon'],
 datetime.date(2025, 8, 12): ['morning', 'afternoon'],
 datetime.date(2025, 8, 13): ['morning', 'afternoon'],
 datetime.date(2025, 8, 14): ['afternoon'],
 datetime.date(2025, 8, 15): ['morning', 'afternoon'],
 datetime.date(2025, 8, 18): ['morning', 'afternoon'],
 datetime.date(2025, 8, 19): ['morning', 'afternoon'],
 datetime.date(2025, 8, 20): ['morning', 'afternoon'],
 datetime.date(2025, 8, 21): ['afternoon'],
 datetime.date(2025, 8, 22): ['morning', 'afternoon'],
 datetime.date(2025, 8, 25): ['morning', 'afternoon'],
 datetime.date(2025, 8, 26): ['morning', 'afternoon'],
 datetime.date(2025, 8, 27): ['morning', 'afternoon'],
 datetime.date(2025, 8, 28): ['afternoon'],
 dat

**We're ready to instantiate a new instance of the CpModel class from the OR-Tools library. The variable model now holds a specific, usable object that we will shortly add constraints to. We'll also create a dictionary that holds a binary decision variable for each provider for each date, and for each session based on the above August 2025 calendar.**

In [10]:
model = cp_model.CpModel()
shift_vars = create_shift_variables(model, list(config['providers'].keys()), calendar)

In [11]:
# For each possible clinic session in August 2025, there's a binary decision variable (0 or 1) that will be solved by the model
shift_vars['Orcutt']

{datetime.date(2025, 8, 4): {'morning': Orcutt_2025-08-04_morning(0..1),
  'afternoon': Orcutt_2025-08-04_afternoon(0..1)},
 datetime.date(2025, 8, 5): {'morning': Orcutt_2025-08-05_morning(0..1),
  'afternoon': Orcutt_2025-08-05_afternoon(0..1)},
 datetime.date(2025, 8, 6): {'morning': Orcutt_2025-08-06_morning(0..1),
  'afternoon': Orcutt_2025-08-06_afternoon(0..1)},
 datetime.date(2025, 8, 7): {'afternoon': Orcutt_2025-08-07_afternoon(0..1)},
 datetime.date(2025, 8, 8): {'morning': Orcutt_2025-08-08_morning(0..1),
  'afternoon': Orcutt_2025-08-08_afternoon(0..1)},
 datetime.date(2025, 8, 11): {'morning': Orcutt_2025-08-11_morning(0..1),
  'afternoon': Orcutt_2025-08-11_afternoon(0..1)},
 datetime.date(2025, 8, 12): {'morning': Orcutt_2025-08-12_morning(0..1),
  'afternoon': Orcutt_2025-08-12_afternoon(0..1)},
 datetime.date(2025, 8, 13): {'morning': Orcutt_2025-08-13_morning(0..1),
  'afternoon': Orcutt_2025-08-13_afternoon(0..1)},
 datetime.date(2025, 8, 14): {'afternoon': Orcutt_2

**Now we add the constraints.**

In [12]:
objective_terms = []

In [13]:
add_leave_constraints(model, shift_vars, leave_df)
add_inpatient_block_constraints(model, shift_vars, inpatient_starts_df, inpatient_days_df)
add_clinic_count_constraints(model, shift_vars, config['providers'], inpatient_starts_df)
add_rdo_constraints(model, shift_vars, leave_df, inpatient_days_df, config['clinic_rules'], config['providers'])
add_min_max_staffing_constraints(model, shift_vars, calendar, config['clinic_rules'])

In [14]:
objective_terms.extend(add_clinic_count_constraints(model, shift_vars, config['providers'], inpatient_starts_df))

In [15]:
if objective_terms:
    model.Minimize(sum(objective_terms))

**Finally we solve the model and print the output.**

In [16]:
solver = cp_model.CpSolver()

# Set random seed for reproducibility
solver.parameters.random_seed = 42

status = solver.Solve(model)
solver_wall_time = solver.wall_time

In [17]:
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print(f"Solution found in {solver_wall_time/1000:.6f} seconds") # Convert from ms to seconds
    for day in calendar:
        day_of_week = day.strftime('%A')
        for session in calendar[day]:
            scheduled = [
                provider for provider in shift_vars
                if day in shift_vars[provider]
                and session in shift_vars[provider][day]
                and solver.Value(shift_vars[provider][day][session]) == 1
            ]
            print(f"{day} {day_of_week} {session}: staffed by {scheduled}")
else:
    print("No feasible solution found.")

Solution found in 0.000013 seconds
2025-08-04 Monday morning: staffed by ['Mccrae', 'Miles', 'Selig']
2025-08-04 Monday afternoon: staffed by ['Bornstein', 'Mccrae', 'Selig']
2025-08-05 Tuesday morning: staffed by ['Mccrae', 'Stewart', 'Tanay']
2025-08-05 Tuesday afternoon: staffed by ['Mccrae', 'Miles', 'Tanay']
2025-08-06 Wednesday morning: staffed by ['Selig', 'Stewart', 'Tanay']
2025-08-06 Wednesday afternoon: staffed by ['Miles', 'Selig', 'Tanay']
2025-08-07 Thursday afternoon: staffed by ['Bornstein', 'Selig', 'Tanay']
2025-08-08 Friday morning: staffed by ['Bornstein', 'Mccrae', 'Stewart']
2025-08-08 Friday afternoon: staffed by ['Bornstein', 'Mccrae', 'Selig']
2025-08-11 Monday morning: staffed by ['Bornstein', 'Miles', 'Selig']
2025-08-11 Monday afternoon: staffed by ['Bornstein', 'Miles', 'Selig']
2025-08-12 Tuesday morning: staffed by ['Stewart', 'Tanay', 'Wadlin']
2025-08-12 Tuesday afternoon: staffed by ['Stewart', 'Tanay', 'Wadlin']
2025-08-13 Wednesday morning: staffed b

**A clinic schedule for August 2025 was successfully generated that satisfies all constraints. The schedule accommodates approved leave requests while maintaining a minimum staffing level of 3 providers per session. Each provider receives their appropriate number of clinic sessions per week, with reduced sessions following inpatient service weeks. RDOs were assigned according to provider preferences where possible, avoiding weeks with inpatient duties or leave.**